平面ロボットアームの運動学導出

In [1]:
import sympy as sy
from sympy import pi, cos, sin, tan
from IPython.display import display
from sympy.printing.pycode import pycode

#q1, q2, q3, q4 = sy.symbols("q1, q2, q3, q4")  # 関節角度
t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")
q4 = sy.Function("q4")
omega1 = sy.Function("omega1")
omega2 = sy.Function("omega2")
omega3 = sy.Function("omega3")
omega4 = sy.Function("omega4")

a1, a2, a3, a4 = sy.symbols("a1, a2, a3, a4")
b1, b2, b3, b4 = sy.symbols("b1, b2, b3, b4")
c1, c2, c3, c4 = sy.symbols("c1, c2, c3, c4")



l1, l2, l3, l4 = sy.symbols("l1, l2, l3, l4")  # リンク長さ
lg1, lg2, lg3, lg4 = sy.symbols("lg1, lg2, lg3, lg4")  # 重心までの長さ
m1, m2, m3, m4 = sy.symbols("m1, m2, m3, m4")  # 質量
Ig1, Ig2, Ig3, Ig4 = sy.symbols("I1, I2, I3, I4")  # 慣性モーメント
g = sy.Symbol("g")  # 重力加速度

def R(q):
    return sy.Matrix([
        [cos(q), -sin(q)],
        [sin(q), cos(q)],
    ])


def HTM(q, x, y):
    return sy.Matrix([
        [cos(q), -sin(q), x],
        [sin(q), cos(q), y],
        [0, 0, 1],
    ])


# ジョイントの位置
x1 = R(q1(t)) * sy.Matrix([[l1, 0]]).T
x2 = R(q1(t)) * sy.Matrix([[l1, 0]]).T + \
    R(q1(t) + q2(t)) * sy.Matrix([[l2, 0]]).T
x3 = R(q1(t)) * sy.Matrix([[l1, 0]]).T +\
    R(q1(t) + q2(t)) * sy.Matrix([[l2, 0]]).T +\
        R(q1(t) + q2(t) + q3(t)) * sy.Matrix([[l3, 0]]).T
x4 = R(q1(t)) * sy.Matrix([[l1, 0]]).T +\
    R(q1(t) + q2(t)) * sy.Matrix([[l2, 0]]).T +\
        R(q1(t) + q2(t) + q3(t)) * sy.Matrix([[l3, 0]]).T +\
            R(q1(t) + q2(t) + q3(t) + q4(t)) * sy.Matrix([[l4, 0]]).T

q = sy.Matrix([[q1(t), q2(t), q3(t), q4(t)]]).T




In [2]:
J1 = x1.jacobian(q)
J2 = x2.jacobian(q)
J3 = x3.jacobian(q)
J4 = x4.jacobian(q)

J1_dot = sy.diff(J1, t)
J2_dot = sy.diff(J2, t)
J3_dot = sy.diff(J3, t)
J4_dot = sy.diff(J4, t)

J_all = [J1, J2, J3, J4]
J_dot_all = [J1_dot, J2_dot, J3_dot, J4_dot]

for i, J in enumerate(J_all):
    J_all[i] = J.subs([
        (sy.Derivative(q1(t),t), b1),
        (sy.Derivative(q2(t),t), b2),
        (sy.Derivative(q3(t),t), b3),
        (sy.Derivative(q4(t),t), b4),
        (q1(t), a1),
        (q2(t), a2),
        (q3(t), a3),
        (q4(t), a4),
    ])

for i, J_dot in enumerate(J_dot_all):
    J_dot_all[i] = J_dot.subs([
        (sy.Derivative(q1(t),t), b1),
        (sy.Derivative(q2(t),t), b2),
        (sy.Derivative(q3(t),t), b3),
        (sy.Derivative(q4(t),t), b4),
        (q1(t), a1),
        (q2(t), a2),
        (q3(t), a3),
        (q4(t), a4),
    ])




In [3]:
f = open('sice_kinema.txt', 'w')
for i, j in enumerate([x1, x2, x3, x4]):
    s = '\nx' + str(i) + '='
    f.write(s)
    f.write(str(j))

for i, j in enumerate(J_all):
    s = '\nJ' + str(i) + '='
    f.write(s)
    f.write(str(j))
for i, j in enumerate(J_dot_all):
    s = '\nJ_dot' + str(i) + '='
    f.write(s)
    f.write(str(j))

f.close()


In [4]:
print(J_dot_all[3])

Matrix([[-b1*l1*cos(a1) - l2*(b1 + b2)*cos(a1 + a2) - l3*(b1 + b2 + b3)*cos(a1 + a2 + a3) - l4*(b1 + b2 + b3 + b4)*cos(a1 + a2 + a3 + a4), -l2*(b1 + b2)*cos(a1 + a2) - l3*(b1 + b2 + b3)*cos(a1 + a2 + a3) - l4*(b1 + b2 + b3 + b4)*cos(a1 + a2 + a3 + a4), -l3*(b1 + b2 + b3)*cos(a1 + a2 + a3) - l4*(b1 + b2 + b3 + b4)*cos(a1 + a2 + a3 + a4), -l4*(b1 + b2 + b3 + b4)*cos(a1 + a2 + a3 + a4)], [-b1*l1*sin(a1) - l2*(b1 + b2)*sin(a1 + a2) - l3*(b1 + b2 + b3)*sin(a1 + a2 + a3) - l4*(b1 + b2 + b3 + b4)*sin(a1 + a2 + a3 + a4), -l2*(b1 + b2)*sin(a1 + a2) - l3*(b1 + b2 + b3)*sin(a1 + a2 + a3) - l4*(b1 + b2 + b3 + b4)*sin(a1 + a2 + a3 + a4), -l3*(b1 + b2 + b3)*sin(a1 + a2 + a3) - l4*(b1 + b2 + b3 + b4)*sin(a1 + a2 + a3 + a4), -l4*(b1 + b2 + b3 + b4)*sin(a1 + a2 + a3 + a4)]])


In [5]:
print(J4)

Matrix([[-l1*sin(q1(t)) - l2*sin(q1(t) + q2(t)) - l3*sin(q1(t) + q2(t) + q3(t)) - l4*sin(q1(t) + q2(t) + q3(t) + q4(t)), -l2*sin(q1(t) + q2(t)) - l3*sin(q1(t) + q2(t) + q3(t)) - l4*sin(q1(t) + q2(t) + q3(t) + q4(t)), -l3*sin(q1(t) + q2(t) + q3(t)) - l4*sin(q1(t) + q2(t) + q3(t) + q4(t)), -l4*sin(q1(t) + q2(t) + q3(t) + q4(t))], [l1*cos(q1(t)) + l2*cos(q1(t) + q2(t)) + l3*cos(q1(t) + q2(t) + q3(t)) + l4*cos(q1(t) + q2(t) + q3(t) + q4(t)), l2*cos(q1(t) + q2(t)) + l3*cos(q1(t) + q2(t) + q3(t)) + l4*cos(q1(t) + q2(t) + q3(t) + q4(t)), l3*cos(q1(t) + q2(t) + q3(t)) + l4*cos(q1(t) + q2(t) + q3(t) + q4(t)), l4*cos(q1(t) + q2(t) + q3(t) + q4(t))]])
